<img src="../../img/land_use_clc.png" alt="" style="width: 100%; border-radius: 20px;"/>

## Objective
ToDo:

why did we create this notebook? why landuse?

## Scope

In this file we process a bird dataframe and merge it with the Corine Land Cover Dataframe to get the land use on the location of a bird sighting. With the functions given in this file it is possible to get the land use on the coordinate, to get the most common land use in a square or to get all land uses per percentage in a square around the coordinate.

The CLC was used because the LUCAS dataframe was used first and gave a much less detailed result. The LUCAS dataframe has points on a map which describe the land use whereas the CLC dataframe has polygons all over the map.

To get the land use with LUCAS the nearest point to the coordinate has to be selected. To do the same with CLC (altough much more precises) the coordinates of the bird dataframe are checked whether they are in any polygon. Then the land use of this polygon is used.

To get the most common land use within a square we create squares around each coordinate. Then each square is checked for the landuses within. Then the most common land uses gets returned.

The probably best solution to get the land use is to get the percentage of landuses in each square. This can be done by checking how much of each polygon is in each square. This is really compute heavy and this is also why there might be a better option:

To accomplish this another aspect of the CLC data is used. The CLC data also has points all over the map which can be used to get the land use. While those points also could be inaccurate if looking for the land use on a coordinate, those specific points inside of the square could be counted and calculated how much of each land use is in each square.

In [1]:
import geopandas as gpd
from shapely.geometry import Point, box
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
df_path = r'D:\\Simon\\Documents\\GP\\data\\master_train.csv'
df = pd.read_csv(df_path, index_col=0, low_memory=False)

In [5]:
df.tail(2)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
2660041,15002282,8,Haubentaucher,2018-02-17,47.512154,9.436332,390,1,0,ch,50kmE4250N2700,0.519785,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.480215,water,14
2660042,15002291,8,Haubentaucher,2018-02-17,47.121205,7.240961,430,1,0,ch,50kmE4100N2650,0.837752,0.0,0.060568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101679,urban,1


In [14]:
clc_path = r'D:\\Simon\\Documents\\GP\\data\\util_files\\CLC_2018\\U2018_CLC2018_V2020_20u1.shp'
clc2 = gpd.read_file(clc_path)

In [ ]:
clc['Code_18'].fillna(999, inplace=True)
clc['Code_18'] = clc['Code_18'].astype(int)
clc.drop(columns=['OBJECTID', 'Remark','Area_Ha','ID','Shape_Leng','Shape_Area'], axis=1, inplace=True)

In [8]:
clc_path = r'D:\\Simon\\Documents\\GP\\data\\util_files\\CLC_2018_EU_Crop\\U2018_CLC2018_V2020_20u1_crop.gpkg'
clc2 = gpd.read_file(clc_path)

In [9]:
clc2['Code_18'].fillna(999, inplace=True)
clc2['Code_18'] = clc2['Code_18'].astype(int)
clc2.drop(columns=['Remark','Area_Ha','ID', 'OBJECTID'], axis=1, inplace=True)

In [10]:
clc2.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

In [11]:
clc2.head(2)

,Code_18,geometry
0,112,"MULTIPOLYGON (((3852601.053 2979540.102, 38526..."
1,112,"MULTIPOLYGON (((3851007.206 2981146.329, 38510..."


In [12]:
clc_4326 = clc2.to_crs("EPSG:4326")

In [13]:
clc_4326.head(2)

,Code_18,geometry
0,112,"MULTIPOLYGON (((3.49281 49.74160, 3.49325 49.7..."
1,112,"MULTIPOLYGON (((3.46882 49.75471, 3.46911 49.7..."


In [14]:
clc_4326.isna().sum()

Code_18     0
geometry    0
dtype: int64

## Get landuse on coordinate

In [ ]:
df.drop(columns=['land_use_coord', 'land_use_coord_numeric'], inplace=True)

In [79]:
df.tail(2)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent
2660041,15002282,8,Haubentaucher,2018-02-17,47.512154,9.436332,390,1,0,ch,50kmE4250N2700,0.519785,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.480215
2660042,15002291,8,Haubentaucher,2018-02-17,47.121205,7.240961,430,1,0,ch,50kmE4100N2650,0.837752,0.0,0.060568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101679


In [80]:
geometry = [Point(lon, lat) for lon, lat in zip(df['coord_lon'], df['coord_lat'])]
# gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:3035")
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

merged_gdf = gpd.sjoin(gdf, clc_4326, how="left", predicate="within")

merged_gdf.drop(columns=['geometry','index_right'], axis=1, inplace=True)

In [50]:
merged_gdf.tail(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,Code_18
2660038,27523548,469,Bergpieper,2022-08-01,46.563896,8.551648,2085,4,0,ch,50kmE4200N2600,0.000000,0.0,0.000000,0.000000,0.580765,0.0,0.0,0.261244,0.005815,0.0,0.0,0.0,0.152176,321.0
2660039,27523686,338,Mittelspecht,2022-10-08,47.383318,7.666533,798,1,0,ch,50kmE4100N2650,0.000000,0.0,0.695808,0.304192,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,242.0
2660040,15002272,123,Bergente,2018-02-17,47.512154,9.436332,390,1,0,ch,50kmE4250N2700,0.519785,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.480215,512.0
2660041,15002282,8,Haubentaucher,2018-02-17,47.512154,9.436332,390,1,0,ch,50kmE4250N2700,0.519785,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.480215,512.0
2660042,15002291,8,Haubentaucher,2018-02-17,47.121205,7.240961,430,1,0,ch,50kmE4100N2650,0.837752,0.0,0.060568,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.101679,112.0


In [81]:
nan_rows = merged_gdf[merged_gdf['Code_18'].isna()]
len(nan_rows)

46

In [82]:
nan_rows.head(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,Code_18
155188,38355619,463,Wiesenpieper,2019-04-03,54.033346,6.626898,0,2,NaN,de,50kmE4100N3400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257889,34581793,463,Wiesenpieper,2018-09-04,54.494140,7.874070,0,0,NaN,de,50kmE4150N3450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257890,34581823,399,Steinschmätzer,2018-09-04,54.052516,6.640945,0,1,NaN,de,50kmE4100N3400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286363,31552365,531,Rohrammer,2018-04-05,55.087171,6.038972,-1,0,NaN,de,50kmE4050N3550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324169,34581853,399,Steinschmätzer,2018-09-05,54.052516,6.640945,0,0,NaN,de,50kmE4100N3400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
layout = go.Layout(
    mapbox=dict(
        style='open-street-map',
        zoom=10), showlegend=False, margin=dict(l=0, r=0, t=0, b=0))
fig = go.Figure(layout=layout)

for i in range(len(nan_rows)):
    # Get bird sighting
    bird_sighting = nan_rows.iloc[i].copy()
    bird_sighting['color'] = '#d00000'
    geometry = gpd.points_from_xy([bird_sighting['coord_lon']], [bird_sighting['coord_lat']])
    gdf_data_point = gpd.GeoDataFrame([bird_sighting], geometry=geometry, crs='EPSG:4326')

    # Add point to map
    fig.add_trace(px.scatter_mapbox(gdf_data_point,
                    lat='coord_lat', lon='coord_lon', size_max=10, color_discrete_sequence=[gdf_data_point.color],
                    hover_name=gdf_data_point['name_species']).data[0])  # Add hover name

# Show the plot
fig.update_layout(mapbox_style='open-street-map', mapbox_zoom=5,
                  mapbox_center={'lat': bird_sighting['coord_lat'], 'lon': bird_sighting['coord_lon']})
fig.show()


In [1]:
nan_rows.head(5)

NameError: name 'nan_rows' is not defined

In [25]:
merged_gdf['Code_18'].fillna(999, inplace=True)

In [26]:
clc_code_to_landuse = {
    111: 'urban', # 1
    112: 'urban',
    121: 'urban',
    122: 'urban',
    123: 'urban',
    124: 'urban',
    131: 'industrial', # 2
    132: 'industrial',
    133: 'industrial',
    141: 'urban', 
    142: 'urban',
    211: 'agriculture', # 3
    212: 'agriculture',
    213: 'agriculture',
    221: 'agriculture',
    222: 'agriculture',
    223: 'agriculture',
    231: 'agriculture',
    241: 'agriculture',
    242: 'agriculture',
    243: 'agriculture',
    244: 'agriculture',
    311: 'forest', # 4
    312: 'forest',
    313: 'forest',
    321: 'grassland', # 5 
    322: 'grassland',
    323: 'mediterranean_vegetation', # 6 
    324: 'shrubland', # 7 
    331: 'coastal', # 8 
    332: 'rocky_area',# 9
    333: 'sparsley_vegetated', # 10
    334: 'burnt_area', # 11
    335: 'glacier', # 12
    411: 'wetlands', # 13
    412: 'wetlands',
    421: 'wetlands',
    422: 'wetlands',
    423: 'wetlands',
    511: 'water', # 14
    512: 'water',
    521: 'water',
    522: 'water',
    523: 'water',
    990: 'unclassified_land', # 15
    995: 'unclassified_water', # 16
    999: 'unclassified' # 17
}
merged_gdf['land_use_coord'] = merged_gdf['Code_18'].map(clc_code_to_landuse)
merged_gdf.drop(columns=['Code_18'], axis=1, inplace=True)

In [27]:
merged_gdf.head(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord
0,29666972,8,Haubentaucher,2018-01-01,53.157760,8.676993,-1,0,NaN,de,50kmE4200N3300,0.191298,0.000000,0.399560,0.000000,0.0,0.021766,0.0,0.0,0.0,0.0,0.0,0.0,0.387377,water
1,29654244,397,Schwarzkehlchen,2018-01-01,53.127639,8.957263,1,2,NaN,de,50kmE4250N3300,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,agriculture
2,29654521,463,Wiesenpieper,2018-01-01,50.850941,12.146953,269,2,NaN,de,50kmE4450N3050,0.405476,0.000000,0.594524,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,urban
3,29666414,8,Haubentaucher,2018-01-01,51.076006,11.038316,157,8,NaN,de,50kmE4350N3100,0.000000,0.004445,0.376111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.619445,water
4,29656211,8,Haubentaucher,2018-01-01,51.389380,7.067282,52,10,NaN,de,50kmE4100N3100,0.362214,0.000000,0.024020,0.211704,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.402062,urban


In [28]:
landuse_to_numerical = {
    'urban': 1,
    'industrial': 2,
    'agriculture': 3,
    'forest': 4,
    'grassland': 5,
    'mediterranean_vegetation': 6,
    'shrubland': 7,
    'coastal': 8,
    'rocky_area': 9,
    'sparsley_vegetated': 10,
    'burnt_area': 11,
    'glacier': 12,
    'wetlands': 13,
    'water': 14,
    'unclassified_land': 15,
    'unclassified_water': 16,
    'unclassified': 17
}
merged_gdf['land_use_coord_numeric'] = merged_gdf['land_use_coord'].map(landuse_to_numerical)

In [29]:
merged_gdf.head(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
0,29666972,8,Haubentaucher,2018-01-01,53.157760,8.676993,-1,0,NaN,de,50kmE4200N3300,0.191298,0.000000,0.399560,0.000000,0.0,0.021766,0.0,0.0,0.0,0.0,0.0,0.0,0.387377,water,14
1,29654244,397,Schwarzkehlchen,2018-01-01,53.127639,8.957263,1,2,NaN,de,50kmE4250N3300,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,agriculture,3
2,29654521,463,Wiesenpieper,2018-01-01,50.850941,12.146953,269,2,NaN,de,50kmE4450N3050,0.405476,0.000000,0.594524,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,urban,1
3,29666414,8,Haubentaucher,2018-01-01,51.076006,11.038316,157,8,NaN,de,50kmE4350N3100,0.000000,0.004445,0.376111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.619445,water,14
4,29656211,8,Haubentaucher,2018-01-01,51.389380,7.067282,52,10,NaN,de,50kmE4100N3100,0.362214,0.000000,0.024020,0.211704,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.402062,urban,1


In [30]:
merged_gdf.value_counts('land_use_coord', dropna=False)

land_use_coord
agriculture           1211268
water                  641127
urban                  203556
forest                 202359
wetlands               189058
grassland              147757
industrial              22706
shrubland               21436
sparsley_vegetated      10726
rocky_area               5078
coastal                  4771
burnt_area                110
unclassified               46
glacier                    45
Name: count, dtype: int64

In [31]:
merged_gdf.land_use_coord.isnull().sum()

0

In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_on_coord_validata_ch.csv'
merged_gdf.to_csv(df_path)

In [ ]:
# write to other csvs
df_path = "D:\Simon\Documents\GP\data\master_validata.csv"
merged_gdf.to_csv(df_path)

### Create squares around each coordinate
This is to determine the most common land use and also the land use in percentage within a square

In [ ]:
# Convert 1km to degrees
km_to_degrees = 1 / 111.0

# Define the size of the square in degrees
square_size_horizontal = 1 * km_to_degrees
square_size_vertical = square_size_horizontal * 1.6667  # To make it a square

# Create a GeoDataFrame with square polygons around each point
geometry = [Point(lon, lat) for lon, lat in zip(df['coord_lon'], df['coord_lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:3035")

# Create squares around each point
squares_gdf = gdf.copy()
squares_gdf['geometry'] = gdf['geometry'].apply(lambda point: box(
    point.x - square_size_vertical/2, 
    point.y - square_size_horizontal/2,
    point.x + square_size_vertical/2, 
    point.y + square_size_horizontal/2
))

In [ ]:
squares_gdf.tail(5)

In [ ]:
squares_gdf.crs

## Get land use percentage 
(better version - old oversion is below in the notebook)

In [ ]:
df.drop(columns=['urban_area_percent','industrial_area_percent','agriculture_area_percent','forest_area_percent','grassland_area_percent','shrubland_area_percent','coastal_area_percent','rocky_area_percent','sparsley_vegetated_area_percent','burnt_area_percent','glacier_area_percent','wetlands_area_percent','water_area_percent'], inplace=True)

In [ ]:
# execution time: 215m (8 Core CPU)
# crs_projected = "EPSG:4326"
crs_projected = "EPSG:3035"

# Transform your data to the projected CRS 
squares_gdf_projected = squares_gdf.to_crs(crs_projected)
clc_projected = clc.to_crs(crs_projected)

# Perform a spatial join between squares_gdf and clc
joined = gpd.sjoin(squares_gdf_projected, clc_projected, how="inner", predicate='intersects')

# Calculate the area of intersection 
joined['intersection_area'] = joined.apply(lambda x: x['geometry'].intersection(clc_projected.loc[x['index_right'], 'geometry']).area, axis=1)

# Group by the index of squares_gdf and Code_18, and sum the intersection areas 
grouped = joined.groupby([joined.index, 'Code_18'])['intersection_area'].sum()

# Calculate the square area - all squares have the same size so we can just take the area of the first square
total_area = squares_gdf_projected.geometry.area.loc[grouped.index.get_level_values(0)].iloc[0]

# Calculate the percentage of each land use type within each square 
percentage_land_use = grouped / total_area

# Unstack the grouped dataframe to get a dataframe where each row corresponds to a square and each column corresponds to a land use type
land_use_df = percentage_land_use.unstack(fill_value=0)

# Join the land use dataframe with squares_gdf 
result = pd.concat([squares_gdf_projected, land_use_df], axis=1)
result.drop(columns=['geometry'], inplace=True)

In [33]:
result.head(3)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
0,29666972,8,Haubentaucher,2018-01-01,53.157760,8.676993,-1,0,NaN,de,50kmE4200N3300,0.191298,0.0,0.399560,0.0,0.0,0.021766,0.0,0.0,0.0,0.0,0.0,0.0,0.387377,water,14
1,29654244,397,Schwarzkehlchen,2018-01-01,53.127639,8.957263,1,2,NaN,de,50kmE4250N3300,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,agriculture,3
2,29654521,463,Wiesenpieper,2018-01-01,50.850941,12.146953,269,2,NaN,de,50kmE4450N3050,0.405476,0.0,0.594524,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,urban,1


In [32]:
result = merged_gdf.copy()

In [ ]:
nan_rows = result[result[112].isna()]
len(nan_rows)

In [34]:
category_dict = {
    111: 'urban_area_percent',
    112: 'urban_area_percent',
    121: 'urban_area_percent',
    122: 'urban_area_percent',
    123: 'urban_area_percent',
    124: 'urban_area_percent',
    131: 'industrial_area_percent',
    132: 'industrial_area_percent',
    133: 'industrial_area_percent',
    141: 'urban_area_percent',
    142: 'urban_area_percent',
    211: 'agriculture_area_percent',
    212: 'agriculture_area_percent',
    213: 'agriculture_area_percent',
    221: 'agriculture_area_percent',
    222: 'agriculture_area_percent',
    223: 'agriculture_area_percent',
    231: 'agriculture_area_percent',
    241: 'agriculture_area_percent',
    242: 'agriculture_area_percent',
    243: 'agriculture_area_percent',
    244: 'agriculture_area_percent',
    311: 'forest_area_percent',
    312: 'forest_area_percent',
    313: 'forest_area_percent',
    321: 'grassland_area_percent',
    322: 'grassland_area_percent',
    323: 'mediterranean_vegetation_area_percent',
    324: 'shrubland_area_percent',
    331: 'coastal_area_percent',
    332: 'rocky_area_percent',
    333: 'sparsley_vegetated_area_percent',
    334: 'burnt_area_percent',
    335: 'glacier_area_percent',
    411: 'wetlands_area_percent',
    412: 'wetlands_area_percent',
    421: 'wetlands_area_percent',
    422: 'wetlands_area_percent',
    423: 'wetlands_area_percent',
    511: 'water_area_percent',
    512: 'water_area_percent',
    521: 'water_area_percent',
    522: 'water_area_percent',
    523: 'water_area_percent',
    990: 'unclassified_land_area_percent',
    995: 'unclassified_water_area_percent',
    999: 'unclassified_area_percent'
}

category_df = result.copy()
for code, category in category_dict.items():
    if code in result.columns:
        if category not in category_df.columns:
            category_df[category] = result[code]
        else:
            category_df[category] += result[code]


columns_to_drop = [col for col in category_dict.keys() if col in category_df.columns]
category_df.drop(columns=columns_to_drop, inplace=True)


In [35]:
nan_rows = category_df[category_df['urban_area_percent'].isna()]
print(nan_rows.shape)
nan_rows.head(4)

(127, 26)


,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
125574,70838383,158,Seeadler,2022-08-02,55.151639,8.648124,0,1,NaN,de,50kmE4200N3550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wetlands,13
126132,70837521,233,Flussuferläufer,2022-08-02,55.112247,8.561053,0,1,NaN,de,50kmE4200N3550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wetlands,13
150515,45087020,463,Wiesenpieper,2020-03-03,55.177401,8.573939,0,12,NaN,de,50kmE4200N3550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,agriculture,3
155188,38355619,463,Wiesenpieper,2019-04-03,54.033346,6.626898,0,2,NaN,de,50kmE4100N3400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclassified,17


In [36]:
category_df.iloc[330499]

id_sighting                              42289691
id_species                                    463
name_species                         Wiesenpieper
date                                   2019-10-05
coord_lat                               51.208738
coord_lon                                8.826114
altitude                                      405
total_count                                    11
atlas_code                                    NaN
country                                        de
eea_grid_id                        50kmE4200N3100
urban_area_percent                            0.0
industrial_area_percent                       0.0
agriculture_area_percent                      1.0
forest_area_percent                           0.0
grassland_area_percent                        0.0
shrubland_area_percent                        0.0
coastal_area_percent                          0.0
rocky_area_percent                            0.0
sparsley_vegetated_area_percent               0.0


In [37]:
cols_to_check = list(set(category_df.columns).intersection(set(category_dict.values())))

all_nan_rows = category_df[cols_to_check].isna().all(axis=1)

category_df.loc[all_nan_rows, cols_to_check] = 0

category_df.loc[all_nan_rows, 'unclassified_area_percent'] = 1

category_df.loc[~all_nan_rows, 'unclassified_area_percent'] = 0

In [38]:
category_df.iloc[330499]

id_sighting                              42289691
id_species                                    463
name_species                         Wiesenpieper
date                                   2019-10-05
coord_lat                               51.208738
coord_lon                                8.826114
altitude                                      405
total_count                                    11
atlas_code                                    NaN
country                                        de
eea_grid_id                        50kmE4200N3100
urban_area_percent                            0.0
industrial_area_percent                       0.0
agriculture_area_percent                      1.0
forest_area_percent                           0.0
grassland_area_percent                        0.0
shrubland_area_percent                        0.0
coastal_area_percent                          0.0
rocky_area_percent                            0.0
sparsley_vegetated_area_percent               0.0


In [39]:
nan_rows3 = category_df[category_df['unclassified_area_percent'].isna()]
print(nan_rows3.shape)
nan_rows3.head(2)

(0, 27)


,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent


In [40]:
category_df.head(3)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent
0,29666972,8,Haubentaucher,2018-01-01,53.157760,8.676993,-1,0,NaN,de,50kmE4200N3300,0.191298,0.0,0.399560,0.0,0.0,0.021766,0.0,0.0,0.0,0.0,0.0,0.0,0.387377,water,14,0.0
1,29654244,397,Schwarzkehlchen,2018-01-01,53.127639,8.957263,1,2,NaN,de,50kmE4250N3300,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,agriculture,3,0.0
2,29654521,463,Wiesenpieper,2018-01-01,50.850941,12.146953,269,2,NaN,de,50kmE4450N3050,0.405476,0.0,0.594524,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,urban,1,0.0


In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_percentage.csv'
category_df.to_csv(df_path)

In [41]:
# write to other csvs
df_path = 'D:\Simon\Documents\GP\data\master_train.csv'
category_df.to_csv(df_path)

## Get most common landuse within square

In [ ]:
# execution time: 132m
merged_squares = gpd.sjoin(squares_gdf, clc, how="left", predicate="intersects")

In [ ]:
merged_squares.head(1)

In [ ]:
merged_squares.reset_index(drop=True, inplace=True)

In [ ]:
common_land_use = merged_squares.groupby(merged_squares.index)['Code_18'].agg(lambda x: x.value_counts().idxmax() if not x.empty and not x.value_counts().empty else None)

common_land_use_df = pd.DataFrame({'Code_18_sq': common_land_use.values}, index=common_land_use.index)

In [ ]:
common_land_use_df.head(2)

In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_most_common.csv'
common_land_use_df.to_csv(df_path)

## Get land use percentage 

In [ ]:
squares_with_clc = gpd.sjoin(clc, squares_gdf, how='inner', predicate='intersects')

# Calculate area of intersection between squares and CLC polygons for each Code_18 within each square
intersection_areas = squares_with_clc.groupby(['index_right', 'Code_18'])['geometry'].apply(lambda x: x.area)
total_areas = intersection_areas.groupby('index_right').sum()

# Calculate percentages of each Code_18 within each square
results = {}
for square_id, group in intersection_areas.groupby(level=0):
    percentages = {}
    for code_18, area in group.items():
        percentages[code_18[1]] = (area / total_areas[square_id])
    results[square_id] = percentages

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results).T.fillna(0)

In [ ]:
results_df.head(2)

In [ ]:
# Reset the index of results_df to make it easier to merge
results_df.reset_index(inplace=True)

# Merge the original DataFrame 'df' with results_df on the common index or a column
merged_df = pd.merge(df, results_df, left_index=True, right_on='index')

In [ ]:
merged_df.head(2)

In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_percentage.csv'
merged_df.to_csv(df_path)

In [ ]:
common_land_use_df.head(3)

In [ ]:
# Merge the common land use DataFrame with the original DataFrame based on the index
merged_gdf = merged_squares.merge(common_land_use_df, left_index=True, right_index=True)
merged_gdf.drop(columns=['geometry'], axis=1, inplace=True)

In [ ]:
merged_gdf.head(3)

In [ ]:
clc_code_to_numerical_label = {
    111: 'urban', # 1
    112: 'urban',
    121: 'urban',
    122: 'urban',
    123: 'urban',
    124: 'urban',
    131: 'industrial', # 2
    132: 'industrial',
    133: 'industrial',
    141: 'urban', 
    142: 'urban',
    211: 'agriculture', # 3
    212: 'agriculture',
    213: 'agriculture',
    221: 'agriculture',
    222: 'agriculture',
    223: 'agriculture',
    231: 'agriculture',
    241: 'agriculture',
    242: 'agriculture',
    243: 'agriculture',
    244: 'agriculture',
    311: 'forest', # 4
    312: 'forest',
    313: 'forest',
    321: 'grassland', # 5 
    322: 'grassland',
    323: 'mediterranean_vegetation', # 6 
    324: 'shrubland', # 7 
    331: 'coastal', # 8 
    332: 'rocky_area',# 9
    333: 'sparsley_vegetated', # 10
    334: 'burnt_area', # 11
    335: 'glacier', # 12
    411: 'wetlands', # 13
    412: 'wetlands',
    421: 'wetlands',
    422: 'wetlands',
    423: 'wetlands',
    511: 'water', # 14
    512: 'water',
    521: 'water',
    522: 'water',
    523: 'water',
    990: 'unclassified_land', # 15
    995: 'unclassified_water', # 16
    999: 'unclassified' # 17
}
merged_gdf['land_use_coord'] = merged_gdf['Code_18'].map(clc_code_to_numerical_label)
merged_gdf['land_use_sq'] = merged_gdf['Code_18_sq'].map(clc_code_to_numerical_label)

In [ ]:
merged_gdf.head(3)

In [ ]:
merged_gdf['atlas_code'].fillna(0, inplace=True)

In [ ]:
merged_gdf.drop(columns=['Code_18', 'Code_18_sq'], axis=1, inplace=True)

In [ ]:
df_path = 'D:\Simon\Documents\GP\data\datasets\selected_species_50km_luse.csv'
merged_gdf.to_csv(df_path)